In [ ]:
# This code downloads the price data for a bunch of cryptoassets and exports it to a .csv
# - To change the start-date modify the start date but keep it in YYYY-MM-DD
# - To change the symbols add more with the same format. 
# - The column labels in the export will correspond to the right column of the
#       coin_symbols object. If you want to change that, modify the list.

In [1]:
import numpy as np
import pandas as pd

from coinmetrics.api_client import CoinMetricsClient

In [2]:
# Start date for the API requests
start_date = '2020-12-31'

# Tokens to download in addition to btc.
coin_symbols = {'eth'   : 'Ethereum',
                'bnb'   : 'Binance',
                'xrp'   : 'Ripple',
                'luna'  : 'Terra',
                'sol'   : 'Solana',
                'ada'   : 'Cardano',
                'avax'  : 'Avalanche',
                'dot'   : 'Polkadot',
                'doge'  : 'Dogecoin',
                'shib'  : 'Shiba Inu',
                'matic' : 'Polygon',
                'cro'   : 'Cronos',
                'atom'  : 'Cosmos',
                'ltc'   : 'Litecoin',
                'near'  : 'NEAR',
                'link'  : 'Chainlink',
                'uni'   : 'Uniswap',
                'trx'   : 'TRON',
                'bch'   : 'Bitcoin Cash',
                'algo'  : 'Algorand',
                'mana'  : 'Decentraland',
                'ftm'   : 'Fantom',
                'etc'   : 'Ethereum Classic',
                'sand'  : 'The Sandbox',
                'fil'   : 'Filecoin'
                }

In [3]:
# Download the data for bitcoin
client = CoinMetricsClient()
btc_data = client.get_asset_metrics(
    assets=['btc'],
    metrics=['ReferenceRate'],
    start_time=f'{start_date}T00:00:00Z',
    frequency='1d'
)
df = btc_data.to_dataframe()

df['Bitcoin'] = df['ReferenceRate']
# Extract the price by duplicating so date remains the first column

df.drop(columns=['asset', 'ReferenceRate'], axis=1, inplace=True)
# Drop unwanted columns


In [4]:

# Download all the data
cm_data = client.get_asset_metrics(
    assets=list(coin_symbols.keys()),
    metrics=['ReferenceRate'],
    start_time=f'{start_date}T00:00:00Z',
    frequency='1d'
)
cm_df = cm_data.to_dataframe()

# Iterate through each asset and extract the data
for key in list(coin_symbols.keys()):
    tmp_df = cm_df[cm_df['asset'] == key]
    
    tmp_df = tmp_df.drop(columns=['asset'], axis=1)
    # Drop the unwatned column

    tmp_df = tmp_df.rename(columns={"ReferenceRate": coin_symbols[key]})
    # Rename 4pm ReferenceRate to Ethereum 
    
    df = df.merge(tmp_df, on="time", how="outer")
    # Merge the dataframes


df = df.sort_values(by="time")

In [5]:
df.tail()

,time,Bitcoin,Ethereum,Binance,Ripple,Terra,Solana,Cardano,Avalanche,Polkadot,...,Chainlink,Uniswap,TRON,Bitcoin Cash,Algorand,Decentraland,Fantom,Ethereum Classic,The Sandbox,Filecoin
668,2022-10-30 00:00:00+00:00,20803.899921,1618.894969,303.66351,0.469741,0.000237,32.833754,0.418614,18.28173,6.624128,...,7.584716,7.053579,0.063897,118.295764,0.338047,0.658964,0.224444,25.837278,0.808404,5.661571
669,2022-10-31 00:00:00+00:00,20616.703377,1589.51055,313.238959,0.457816,0.000249,32.844036,0.405295,18.224108,6.639513,...,7.780145,6.907197,0.063103,116.121779,0.358216,0.671328,0.237256,24.498394,0.862626,5.382671
670,2022-11-01 00:00:00+00:00,20490.858181,1571.203871,327.150616,0.464308,0.000242,32.642832,0.406141,19.349504,6.631846,...,7.848572,6.954816,0.063298,115.117368,0.358643,0.673488,0.238807,24.265415,0.875417,5.433654
671,2022-11-02 00:00:00+00:00,20483.966531,1580.255947,324.794674,0.46478,0.00022,32.265686,0.401166,18.680733,6.472968,...,7.675544,7.134424,0.062824,115.040166,0.359818,0.652653,0.232074,24.047603,0.864294,5.37045
672,2022-11-03 00:00:00+00:00,20148.284554,1519.577437,320.112796,0.451286,0.000212,30.781793,0.385535,17.891014,6.237969,...,7.424968,7.089907,0.06152,113.451492,0.362017,0.622044,0.220165,22.926168,0.810603,5.219946


In [6]:
df.to_csv('crypto-prices.csv', index = True)